In [7]:
import re
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from scipy.stats import mannwhitneyu
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
import nilearn.plotting as nplt
import pickle
from .node_estimate import Node
from .timewindow import TimeWindow, sliding_window
from .parcellation import freesurf_dict

In [8]:
root= './'

tree = os.walk(os.path.join(root, 'Source'))

raw_files = []
src_files = []
inv_files = []
bem_files = []
fwd_files = []
trans_files = []
epochs_files = []
ave_files = []
stc_files = []

subjects_found = False

for walk in tree:
    for file in walk[2]:
        if re.search(r'.*raw\.fif', file):
            raw_files.append(os.path.join(walk[0], file))
        if re.search(r'.*src.*\.fif', file):
            src_files.append(os.path.join(walk[0], file))
        if re.search(r'.*inv.*\.fif', file):
            inv_files.append(os.path.join(walk[0], file))
        if re.search(r'.*bem.*\.fif', file):
            bem_files.append(os.path.join(walk[0], file))
        if re.search(r'.*fwd.*\.fif', file):
            fwd_files.append(os.path.join(walk[0], file))
        if re.search(r'.*trans.*\.fif', file):
            trans_files.append(os.path.join(walk[0], file))
        if re.search(r'.*epo.*\.fif', file):
            epochs_files.append(os.path.join(walk[0], file))
        if re.search(r'.*ave.*\.fif', file):
            ave_files.append(os.path.join(walk[0], file))
        if re.search(r'.*stc.*\.fif.*', file):
            stc_files.append(os.path.join(walk[0], file))

    for subdir in walk[1]:
        if subdir == 'Subjects' or subdir == 'subjects' and not subjects_found:
            subjects_found = True
            subjects_dir = os.path.join(walk[0], subdir)
        elif subdir == 'Subjects' or subdir == 'subjects' and subjects_found:
            raise OSError("There are two subjects directories: {}, {}; Only one must be".format(
                subjects_dir, os.path.join(walk[0], subdir)
            ))

if not raw_files:
    raise OSError("No one of raw files are found. Raw file must have extension \'.fif\' and ends with \'raw\'")

if not subjects_found:
    raise OSError("Subjects directory not found!")
subjects = os.listdir(subjects_dir)

subject_dirs = []

for subject in subjects:
    subject_dirs.append(os.path.join(subjects_dir, subject))

raw_file = raw_files[0]
bem_file = bem_files[0]
src_file = src_files[0]
fwd_file = fwd_files[0]
trans_file = trans_files[0]

res_folder = os.path.join(root, 'Pipeline', subjects[0])

res_raw_folder = os.path.join(res_folder, 'Raw')
res_bem_folder = os.path.join(res_folder, 'Bem')
res_src_folder = os.path.join(res_folder, 'Src')
res_fwd_folder = os.path.join(res_folder, 'Fwd')
res_events_folder = os.path.join(res_folder, 'Events')
res_epochs_folder = os.path.join(res_folder, 'Epochs')
res_evoked_folder = os.path.join(res_folder, 'Evoked')
res_cov_folder = os.path.join(res_folder, 'Cov')
res_inv_folder = os.path.join(res_folder, 'Inv')
res_sLORETA_folder = os.path.join(res_folder, 'sLORETA')
res_nodes_folder = os.path.join(res_folder, 'NodesEstimate')

res_raw_file = os.path.join(res_raw_folder, 'raw.fif')
res_bem_file = os.path.join(res_bem_folder, 'raw_bem.fif')
res_src_file = os.path.join(res_src_folder, 'raw_src_ico5.fif')
res_fwd_file = os.path.join(res_fwd_folder, 'raw_fwd_ico5.fif')
res_events_file = os.path.join(res_events_folder, 'raw_eve.fif')
res_epochs_file = os.path.join(res_epochs_folder, 'raw_epo.fif')
res_evoked_file = os.path.join(res_evoked_folder, 'raw_ave.fif')
res_cov_file = os.path.join(res_cov_folder, 'noise_cov.fif')
res_inv_file = os.path.join(res_inv_folder, 'raw_inv.fif')
res_sLORETA_file = os.path.join(res_sLORETA_folder, 'sLORETA_raw_ave_inv.fif')
res_sLORETA_file_lh = os.path.join(res_sLORETA_folder, 'sLORETA_raw_ave_inv.fif-lh.stc')
res_sLORETA_file_rh = os.path.join(res_sLORETA_folder, 'sLORETA_raw_ave_inv.fif-rh.stc')
res_nodes_strength_file = os.path.join(res_nodes_folder, 'nodes_strength_auc.dat')
res_nodes_file = os.path.join(res_nodes_folder, 'nodes.pkl')

subject_dir = subject_dirs[0]
subject = subjects[0]

In [9]:
regexp = r'(MEG [12][45][123]1|EEG 00.)'
conductivity = (0.3,)  # for single layer
# conductivity = (0.3, 0.006, 0.3)  # for three layers
epochs_tmin, epochs_tmax = -3, 3
crop_time = 120
snr = 0.5  # use SNR smaller than 1 for raw data
lambda2 = 1.0 / snr ** 2
method = "sLORETA"
nfreq = 50
lfreq = 1
hfreq = 70

In [10]:
def mkdir(path):

    try:
        os.mkdir(path)

    except OSError:
        print("PIPELINE: creation of the directory %s failed" % path)

    else:
        print("PIPELINE: successfully created the directory %s " % path)


@sliding_window(size=1200, overlap=0.5)
def do_nothing(sig):

    return sig


@sliding_window(1200, 0.5)
def pearson(signals):

    nsigmals, lsignals = signals.shape
    out = np.zeros((nsigmals, nsigmals))

    for i in range(nsigmals):
        for j in range(nsigmals):
            out[i, j] = np.corrcoef(signals[i, :], signals[j, :])[0, 1]

    return out


def mean_across_tw(twlist):

    l, w = twlist[0].data.shape
    voxel = voxel_from_tw(twlist)
    out = np.zeros((l, w))

    for i in range(w):
        for j in range(l):
            out[i, j] = np.mean(voxel[i, j, :])

    return out


def voxel_from_tw(twlist):

    l, w = twlist[0].data.shape
    h = len(twlist)
    voxel = np.zeros((l, w, h))

    for i in range(h):
        voxel[:, :, i] = twlist[i].data

    return voxel


def notchfir(raw, lfreq, nfreq, hfreq):

    meg_picks = mne.pick_types(raw.info, meg=True, eeg=False, eog=False)
    raw_filtered = raw \
        .load_data() \
        .notch_filter(nfreq, meg_picks) \
        .filter(l_freq=lfreq, h_freq=hfreq)

    return raw_filtered


def artifacts_clean(raw):

    ica = ICA(n_components=15, random_state=97)
    ica.fit(raw)
    ica.exclude = ica.find_bads_eog(raw)[0] + \
                  ica.find_bads_ecg(raw, method='correlation', threshold=3.0)[0]

    ica.apply(raw)

    return raw


def first_processing(raw):

    raw_cropped = raw.crop(tmax=crop_time)
    raw_filtered = notchfir(raw_cropped, lfreq, nfreq, hfreq)
    raw_reconstructed = artifacts_clean(raw_filtered)

    del raw_filtered

    return raw_reconstructed


def nodes_strength(label_tc):

    pearson_matrices = pearson(label_tc)
    pears_mean = mean_across_tw(pearson_matrices)
    n_strength = np.array([])

    for i in range(pears_mean.shape[0]):
        n_strength = np.append(n_strength, np.sum(pears_mean[i, :]))

    return n_strength


def Drs_norm(Drs, Drs_max, Drs_min):

    return (Drs -Drs_min)/(Drs_max - Drs_min)

In [11]:
if not os.path.exists('./Pipeline'):
    mkdir('./Pipeline')

if not os.path.exists(res_folder):
    mkdir(res_folder)

PIPELINE: successfully created the directory ./Pipeline 
PIPELINE: successfully created the directory ./Pipeline/B1C2 


In [12]:
if os.path.isfile(res_raw_file):
    raw = mne.io.read_raw_fif(res_raw_file)

elif os.path.isfile(raw_file):
    raw = mne.io.read_raw_fif(raw_file)
    raw = first_processing(raw)
    path = res_raw_folder

    mkdir(path)

    raw.save(res_raw_file)

else:
    raise OSError('PIPELINE: Raw-file not found')

Opening raw data file ./Source/Source/B1C2_ii_run1_raw_tsss_mc_art_corr_raw.fif...
    Range : 39000 ... 1257999 =     39.000 ...  1257.999 secs
Ready.
Current compensation grade : 0
Reading 0 ... 120000  =      0.000 ...   120.000 secs...
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 6601 samples (6.601 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) metho

In [13]:
if os.path.isfile(res_src_file):
    src = mne.read_source_spaces(res_src_file)

elif os.path.isfile(src_file):
    src = mne.read_source_spaces(src_file)
    path = res_src_folder

    mkdir(path)

    src.save(res_src_file)

else:
    print('PIPELINE: Source spaces not found, creating a new one...')
    src = mne.setup_source_space(subject, spacing='ico5', add_dist='patch', subjects_dir=subject_dir)
    path = res_src_folder

    mkdir(path)

    src.save(res_src_file)

<ipython-input-13-b389057533a4>:5: RuntimeWarning: This filename (./Source/Source/source_spaces_src_oct5.fif) does not conform to MNE naming conventions. All source space files should end with -src.fif, -src.fif.gz, _src.fif, _src.fif.gz, -fwd.fif, -fwd.fif.gz, _fwd.fif, _fwd.fif.gz, -inv.fif, -inv.fif.gz, _inv.fif or _inv.fif.gz
  src = mne.read_source_spaces(src_file)


AttributeError: 'NoneType' object has no attribute 'kind'

In [ ]:
if os.path.isfile(res_bem_file):
    bem = mne.read_bem_solution(res_bem_file)

elif os.path.isfile(bem_file):
    bem = mne.read_bem_solution(bem_file)
    path = res_bem_folder

    mkdir(path)

    mne.write_bem_solution(res_bem_file, bem)

else:
    print('PIPELINE: BEM-surface not found, creating a new one...')
    model = mne.make_bem_model(subject=subject, ico=4, conductivity=conductivity, subjects_dir=subject_dir)
    bem = mne.make_bem_solution(model)
    path = res_bem_folder

    mkdir(path)

    mne.write_bem_solution(res_bem_file, bem)

In [ ]:
if os.path.isfile(res_fwd_file):
    fwd = mne.read_forward_solution(res_fwd_file)

elif os.path.isfile(fwd_file):
    fwd = mne.read_forward_solution(fwd_file)
    path = res_fwd_folder

    mkdir(path)

    mne.write_forward_solution(res_fwd_file, fwd)

else:
    print('PIPELINE: Forward solution not found, creating a new one...')
    fwd = mne.make_forward_solution(res_raw_file, trans=trans_file, src=src, bem=bem, meg=True, eeg=False,
                                    mindist=5.0, n_jobs=1, verbose=True)
    path = res_fwd_folder

    mkdir(path)

    mne.write_forward_solution(res_fwd_file, fwd)

In [ ]:
events = np.array([[
        raw.first_samp + raw.time_as_index(crop_time/2 - 30)[0],
        0,
        1
    ]])

In [ ]:
raw.plot(events=events, start=0, duration=120, color='gray', event_color={1: 'r'})

In [ ]:
if os.path.isfile(res_epochs_file):
    epochs = mne.read_epochs(res_epochs_file)

else:
    print('PIPELINE: Epochs not found, creating a new one...')
    epochs = mne.Epochs(raw, events, tmin=epochs_tmin, tmax=epochs_tmax,
                        preload=True)
    path = res_epochs_folder

    mkdir(path)

    epochs.save(res_epochs_file)

In [ ]:
epochs.plot()

In [ ]:
if os.path.isfile(res_evoked_file):
    evoked = mne.read_evokeds(res_evoked_file)
else:
    print('PIPELINE: Evokeds not found, creating a new one...')
    evoked = epochs.average()
    path = res_evoked_folder

    mkdir(path)

    mne.write_evokeds(res_evoked_file, evoked)

    evoked = [evoked]

In [ ]:
evoked[0].plot()

In [ ]:
if os.path.isfile(res_cov_file):
    noise_cov = mne.read_cov(res_cov_file)

else:
    print('PIPELINE: Noise covariance not found, creating a new one...')
    noise_cov = mne.compute_covariance(epochs.copy().pick_types(meg=True, eeg=False, eog=False), tmin=epochs_tmin, tmax=0,
                                       method='empirical')
    path = res_cov_folder

    mkdir(path)

    mne.write_cov(res_cov_file, noise_cov)

In [ ]:
if os.path.isfile(res_inv_file):
    inv = mne.minimum_norm.read_inverse_operator(res_inv_file)

else:
    print('PIPELINE: Inverse operator not found, creating a new one...')
    inv = mne.minimum_norm.make_inverse_operator(raw.info, fwd, noise_cov)
    path = res_inv_folder

    mkdir(path)

    mne.minimum_norm.write_inverse_operator(res_inv_file, inv)

In [ ]:
if os.path.isfile(res_sLORETA_file_lh) and os.path.isfile(res_sLORETA_file_rh):
    stc = mne.read_source_estimate(res_sLORETA_file_lh)
    stc.__add__(mne.read_source_estimate(res_sLORETA_file_rh))

else:
    print('PIPELINE: sLORETA not found, creating a new one...')
    stc = mne.minimum_norm.apply_inverse(evoked[0], inv, lambda2, 'sLORETA')
    path = res_sLORETA_folder

    mkdir(path)

    stc.save(res_sLORETA_file)

In [ ]:
# kwargs = dict(initial_time=0.08, hemi='both', subjects_dir=subject_dir,
#               size=(600, 600), subject=subject)
# brain = stc.plot(figure=1, **kwargs)

In [ ]:
labels = mne.read_labels_from_annot(subject, parc='aparc', subjects_dir=subject_dir)

label_tc = stc.extract_label_time_course(labels, src=inv['src'], mode='mean_flip')

In [ ]:
if os.path.isfile(res_nodes_strength_file):
    n_strength = np.fromfile(res_nodes_strength_file,
                      dtype=float)

else:
    print('PIPELINE: Node strength not found, compute a new one')
    n_strength = nodes_strength(label_tc)
    path = res_nodes_folder

    mkdir(path)

    n_strength.tofile(res_nodes_strength_file)

In [ ]:
# nodes strength

# plt.plot(n_strength, 'o')
# plt.title('Node Strength')
# plt.xlabel('node: number')
# plt.ylabel('node: strength')
# plt.show()

In [ ]:
n_strength_sorted = np.sort(n_strength)
n_strength_sorted_rev = np.flip(n_strength_sorted)
resected_nodes = 15

Drs_max = np.sum(n_strength_sorted_rev[0:resected_nodes])/np.sum(n_strength_sorted[resected_nodes + 1:len(n_strength)-1])
Drs_min = np.sum(n_strength_sorted[0:resected_nodes])/np.sum(n_strength_sorted[resected_nodes + 1:len(n_strength)-1])
Drs = np.sum(n_strength[0:resected_nodes])/np.sum(n_strength_sorted[resected_nodes + 1:len(n_strength)-1])

u_stat_max, p_val_max = mannwhitneyu(n_strength_sorted_rev[0:resected_nodes],
                                 n_strength_sorted_rev[resected_nodes + 1:len(n_strength)-1])
u_stat_min, p_val_min = mannwhitneyu(n_strength_sorted[0:resected_nodes],
                                 n_strength_sorted[resected_nodes + 1:len(n_strength)-1])
u_stat, p_val = mannwhitneyu(n_strength[0:resected_nodes],
                             n_strength[resected_nodes + 1:len(n_strength)-1])


In [ ]:
# Node Strength, Drs; First n values
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(n_strength[0:resected_nodes], 'o', color='m')
ax1.set_title('removed')
ax2.plot(n_strength[resected_nodes + 1:len(n_strength)-1], 'o', color='b')
fig.suptitle('Node Strength, Drs = {}, u-stat = {}'.format(Drs_norm(Drs, Drs_max, Drs_min), u_stat))
ax2.set_title('spared')
fig.show()

# Node Strength, Drs; Resected above spared
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(n_strength_sorted_rev[0:resected_nodes], 'o', color='m')
ax1.set_title('removed')
ax2.plot(n_strength_sorted_rev[resected_nodes + 1:len(n_strength)-1], 'o', color='b')
fig.suptitle('Node Strength, Drs = {}, u-stat = {}'.format(Drs_norm(Drs_max, Drs_max, Drs_min), u_stat_max))
ax2.set_title('spared')
fig.show()

# Node Strength, Drs; Spared above resected
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(n_strength_sorted[0:resected_nodes], 'o', color='m')
ax1.set_title('removed')
ax2.plot(n_strength_sorted[resected_nodes + 1:len(n_strength)-1], 'o', color='b')
fig.suptitle('Node Strength, Drs = {}, u-stat = {}'.format(Drs_norm(Drs_min, Drs_max, Drs_min), u_stat_min))
ax2.set_title('spared')
fig.show()

del n_strength_sorted, n_strength_sorted_rev

In [ ]:
# import sklearn.metrics as sm
# arr = n_strength
#
# for i in range(arr.shape[0]):
#     arr[i] = int(np.round(arr[i]))
#
# print(sm.roc_auc_score(arr[0:resected_nodes], arr[resected_nodes+1:], multi_class='ovo'))

In [ ]:
# to show one node

# coordinates = np.array([0])
# node_str = np.zeros(coordinates.shape[0])
# nplt.plot_markers(np.array([0, 0]), np.array([
#     freesurf_dict['bankssts-lh'],
#     np.array([1000, 1000, 1000]) ## plot markers does not work with one node
# ]))
# nplt.show()

In [ ]:
# to show all of nodes from freesurf_dict

# coord_from_dict = list(freesurf_dict.values())
# del coord_from_dict[0], coord_from_dict[34]
# coordinates = np.array(coord_from_dict)
# node_str = np.zeros(coordinates.shape[0])
# nplt.plot_markers(node_str, coordinates)
# nplt.show()

In [ ]:
# to show nodes in label pos coordinates

arr = []
for label in labels:
    arr.append(label.pos.mean(axis=0)*1000)
coordinates = np.array(arr)
nplt.plot_markers(n_strength, coordinates, node_cmap='black_red_r')
nplt.show()

In [ ]:
if os.path.isfile(res_nodes_file):
    print('Reading nodes...')
    nodes = pickle.load(open(res_nodes_file, 'rb'))

else:
    print('PIPELINE: Nodes file not found, create new one')

    if not os.path.exists(res_nodes_folder):
        mkdir(res_nodes_folder)

    nodes = []

    for i in range(len(n_strength)):
        nodes.append(Node(n_strength[i], label_tc[i, :], labels[i]))

    pickle.dump(nodes, open(res_nodes_file, 'wb'))

coordinates = []
for node in nodes:
    coordinates.append(node.nilearn_coordinates)

nplt.plot_markers(n_strength, coordinates, node_cmap='black_red_r')
nplt.show()

In [ ]:
del nodes, stc, src, raw, fwd, arr, coordinates,\
    n_strength, bem, labels, label_tc, inv, noise_cov